In [14]:
# Import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [15]:
r=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [16]:
r.status_code

200

In [18]:
soup = BeautifulSoup(r.text,'html.parser')

In [19]:
tbodys = soup.find('tbody')

In [20]:
#table header capture
header=tbodys.find_all('th')
header=[x.text.strip() for x in header]

In [21]:
rows = tbodys.find_all('tr')
table=[]
for row in rows[1:]:
    cols=row.find_all('td')
    cols=[x.text.strip() for x in cols]
    table.append(cols)


In [22]:
df=pd.DataFrame(table,columns=header)

* drop missing borough

In [23]:
df=df[~(df['Borough']=='Not assigned')] 

In [24]:
postcode=df.Postcode
borough=df.Borough
Neighbourhood=df.Neighbourhood

* Missing neighbourhood replaced by borough

In [25]:
for i in range(len(borough)):
    if Neighbourhood.iloc[i]=='Not assigned':
        Neighbourhood.iloc[i]=borough.iloc[i]

In [26]:
postcode=df.Postcode.tolist()
borough=df.Borough.tolist()
Neighbourhood=df.Neighbourhood.tolist()

* Neighbourhood list groupby postcode

In [27]:
new_table=[]
postcode_explored = [] # Initializing the list of explored postcodes
for index_i, postcode_i in enumerate(postcode) :   
    if postcode_i not in postcode_explored :
        nbds = Neighbourhood[index_i]
        for index_f, postcode_f in enumerate(postcode) :
            if postcode_i == postcode_f and index_i != index_f:
                nbds = nbds + ', ' + Neighbourhood[index_f] # Concatenating the neighbourhood names
        new_table.append([postcode_i, borough[index_i], nbds]) # Writing the rows in the csv file
        postcode_explored.append(postcode_i)

In [28]:
df=pd.DataFrame(new_table,columns=header)

In [29]:
df.shape

(103, 3)

In [30]:
df1=pd.read_csv('https://cocl.us/Geospatial_data')

In [31]:
df1.rename(columns={'Postal Code':'Postcode'}, inplace=True)

In [32]:
lldf=df.merge(df1,on='Postcode')

In [33]:
lldf.shape

(103, 5)

In [34]:
lldf

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509
